# (1)

We are to derive the following two integrals

$$
\ln 2 = \int_0^1 \frac{1}{1+x}dx = \int_0^1 f(x) dx
$$

and

$$
\pi = \int_0^1 \frac{4}{1+x^2}dx = \int_0^1 g(x) dx
$$

Recall the composite Simpson formula, that integrates a function $f(x)$ over the interval $[a,b]$ by first dividing the interval into $n$ subintervals of equal length $h = (b-a)/n$, and then approximating the integral in each interval by the formula

$$
S_i(f) = \int_y^{y+h} f(x) dx = \frac{h}{6} \left[ f(y) + 4f(y+h/2) + f(y+h) \right]
$$

where $y = a + i h$ for $i = 0,1,2,\ldots,n-1$. $S(f) = \sum_{i=0}^{n-1} S_i(f)$ is the composite Simpson formula for the integral of $f(x)$ over $[a,b]$. We can derive that the error of the composite Simpson formula is

$$
I(f) - S(f) = -\frac{h^4}{2880} (b-a) f^{(4)}(\xi), \quad \xi \in [a,b]
$$

We can say roughly the error is $e(h) = \sup |f^{(4)}(x)| (b-a) \cdot h^4/2880$.

In our case both functions have $\sup |f^{(4)}|(x) = 24, b-a=1$, $e=10^{-8}/2$, we can solve to get $h=0.02783$ (h rounded down for control of error).

### Result

In the following code the results are calculated. Results are within the error bounds.

# (2)

The calculation is done very similarly. The error is $e(h) = \sup |f^{(4)}(x)| (b-a) \cdot h^4/4320$. All numerical results required by the problem are printed.


In [1]:
import numpy as np
import sympy, scipy
import math
from sympy.abc import x
f_expr = 1 / (1 + x)
g_expr = 4 / (1 + x * x)
def select_h(f, x, method, a=0, b=1, eps=.5e-8):
    if method == "simpson":
        f4 = sympy.diff(f, x, x, x, x)
        f4_max = sympy.maximum(f4, x, sympy.Interval(a,b))
        f4_min = sympy.minimum(f4, x, sympy.Interval(a,b))
        f4_norm = max(abs(f4_max), abs(f4_min))
        h = (2880 * eps / f4_max / (b - a)) ** (1/4)
    elif method == "gauss":
        f4 = sympy.diff(f, x, x, x, x)
        f4_max = sympy.maximum(f4, x, sympy.Interval(a,b))
        f4_min = sympy.minimum(f4, x, sympy.Interval(a,b))
        f4_norm = max(abs(f4_max), abs(f4_min))
        h = (4380 * eps / f4_max / (b - a)) ** (1/4)
    else:
        raise ValueError("method must be 'simpson' or 'gauss'")
    return h

hf = select_h(f_expr, x, "simpson")
hg = select_h(g_expr, x, "simpson")
print(f"Selected h for f is {hf = }")
print(f"Selected h for g is {hg = }")

Selected h for f is hf = 0.0278315768371374
Selected h for g is hg = 0.0196798967126543


In [2]:
import math

import scipy.integrate
def f(x):
    return 1 / (1 + x)
def g(x):
    return 4 / (1 + x * x)
def comp_simpson(f, a, b, h):
    # weights: (1 4 2 4 2 4 ... 2 4 1) * h / 6, where 4 at halfstep
    n = math.ceil((b - a) / h)
    h = (b - a) / n
    x = np.linspace(a, b, 2 * n + 1)
    # print(x.dtype, n)
    y = f(x)
    return h / 6 * (y[0] + 2 * np.sum(y[2:-2:2]) + 4 * np.sum(y[1:-1:2]) + y[-1])
def comp_gauss(f, a, b, h):
    n = math.ceil((b - a) / h)
    h = (b - a) / n
    x = np.linspace(a, b, n, endpoint=False)
    yl = f(x + h / 2 - h / (2 * np.sqrt(3)))
    yr = f(x + h / 2 + h / (2 * np.sqrt(3)))
    return h / 2 * np.sum(yl + yr)
scipy.integrate.simpson

print(f"{math.log(2) = }")
print(f"{hf = }")
print(f"{comp_simpson(f, 0, 1, hf) = }")
print(f"{comp_simpson(f, 0, 1, hf / 10) = }")

print(f"{math.pi = }")
print(f"{comp_simpson(g, 0, 1, hf) = }")
print(f"{comp_simpson(g, 0, 1, hf / 10) = }")

hf = select_h(f_expr, x, "gauss")
hg = select_h(g_expr, x, "gauss")
print(f"Gauss method, selected h for f is {hf = }")
print(f"Gauss method, selected h for g is {hg = }")
print(f"{comp_gauss(f, 0, 1, hf) = }")
print(f"{comp_gauss(f, 0, 1, hf / 10) = }")
print(f"{comp_gauss(g, 0, 1, hg) = }")
print(f"{comp_gauss(g, 0, 1, hg / 10) = }")


math.log(2) = 0.6931471805599453
hf = 0.0278315768371374
comp_simpson(f, 0, 1, hf) = 0.693147181722225
comp_simpson(f, 0, 1, hf / 10) = 0.693147180560062
math.pi = 3.141592653589793
comp_simpson(g, 0, 1, hf) = 3.14159265358951
comp_simpson(g, 0, 1, hf / 10) = 3.14159265358979
Gauss method, selected h for f is hf = 0.0309070972225982
Gauss method, selected h for g is hg = 0.0218546180328911
comp_gauss(f, 0, 1, hf) = 0.693147179462666
comp_gauss(f, 0, 1, hf / 10) = 0.693147180559827
comp_gauss(g, 0, 1, hg) = 3.14159265358984
comp_gauss(g, 0, 1, hg / 10) = 3.14159265358979
